In [ ]:
# Create the Spark Session
import pyspark
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("Triggers in Spark Streaming") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', f'org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark.__version__}')
    .config("spark.sql.shuffle.partitions", 8)
    .master("spark://spark-master:7077") 
    .getOrCreate()
)

spark

In [ ]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

kafka_df.printSchema()

In [ ]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql import functions as F

kafka_json_df = kafka_df.withColumn("value", F.expr("cast(value as string)"))

In [ ]:
# Schema of the Pyaload

from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = (
    StructType(
    [StructField('customerId', StringType(), True), 
    StructField('data', StructType(
        [StructField('devices', 
                     ArrayType(StructType([ 
                        StructField('deviceId', StringType(), True), 
                        StructField('measure', StringType(), True), 
                        StructField('status', StringType(), True), 
                        StructField('temperature', LongType(), True)
                    ]), True), True)
        ]), True), 
    StructField('eventId', StringType(), True), 
    StructField('eventOffset', LongType(), True), 
    StructField('eventPublisher', StringType(), True), 
    StructField('eventTime', StringType(), True)
    ])
)

In [ ]:
# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", F.from_json(F.col("value"), json_schema)).selectExpr("values_json.*")
streaming_df.printSchema()

In [ ]:
# Lets explode the data as devices contains list/array of device reading
exploded_df = streaming_df.withColumn("data_devices", F.explode("data.devices"))
exploded_df.printSchema()

In [ ]:
# Flatten the exploded df
flattened_df = (
    exploded_df
    .drop("data")
    .withColumn("deviceId", F.col("data_devices.deviceId"))
    .withColumn("measure", F.col("data_devices.measure"))
    .withColumn("status", F.col("data_devices.status"))
    .withColumn("temperature", F.col("data_devices.temperature"))
    .drop("data_devices")
)
flattened_df.printSchema()

In [ ]:
# Running in once/availableNow and processingTime mode
# Write the output to console sink to check the output

(flattened_df
 .writeStream
 .format("console")
 .outputMode("append")
 .trigger(processingTime='10 seconds')
 .option("checkpointLocation", f"/home/jovyan/streaming_checkpoint_dir/{spark.sparkContext.appName.replace(' ', '_')}")
 .start()
 .awaitTermination())

In [ ]:
# View data from Memory Sink
spark.sql("select * from kafka_table").show()